In [1]:
from datetime import date, timedelta
import time

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [2]:
df_train = pd.read_csv(
    '../input/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "../input/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [3]:
items = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")

df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

In [4]:
oil_price = pd.read_csv('../input/oil.csv')
oil_price = oil_price[oil_price.date>='2017-01-01']

In [5]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [6]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

items = items.reindex(df_2017.index.get_level_values(1))

In [7]:
oil_2017 = df_2017.stack().reset_index()

In [8]:
oil_2017['date'] = pd.to_datetime(oil_2017['date'])
oil_price['date'] = pd.to_datetime(oil_price['date'])

In [9]:
oil_2017 = oil_2017.merge(oil_price, on='date', how='left')

In [11]:
oil_2017.fillna(-1)
oil_2017.drop(0, axis=1,inplace=True)

In [17]:
oil_2017 = oil_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["dcoilwtico"]].unstack(
        level=-1).fillna(0)

In [24]:
oil_2017.columns = oil_2017.columns.get_level_values(1)

In [25]:
oil_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0         0.0       52.36       53.26   
          99197            0.0         0.0       52.36       53.26   
          103520           0.0         0.0       52.36       53.26   
          103665           0.0         0.0       52.36       53.26   
          105574           0.0         0.0       52.36       53.26   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          53.77       53.98         0.0         0.0   
          99197          53.77       53.98         0.0         0.0   
          103520         53.77       53.98         0.0         0.0   
          103665         53.77       53.98         0.0         0.0   
          105574         53.77       53.98         0.0         0.0   

date                2017-01-09  2017-01-10     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995          51.95       50.82     ...             0.0   
          99197          51.95       50.82     ...             0.0   
          103520         51.95       50.82     ...             0.0   
          103665         51.95       50.82     ...             0.0   
          105574         51.95       50.82     ...             0.0   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995          49.37       49.07       49.59       48.54   
          99197          49.37       49.07       49.59       48.54   
          103520         49.37       49.07       49.59       48.54   
          103665         49.37       49.07       49.59       48.54   
          105574         49.37       49.07       49.59       48.54   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995          48.81         0.0         0.0       47.59       47.57  
          99197          48.81         0.0         0.0       47.59       47.57  
          103520         48.81         0.0         0.0       47.59       47.57  
          103665         48.81         0.0         0.0       47.59       47.57  
          105574         48.81         0.0         0.0       47.59       47.57  

[5 rows x 227 columns]

In [13]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [14]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "oil_1_2017": get_timespan(oil_2017, t2017, 1, 1).values.ravel(),
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
            
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "std_3_2017": get_timespan(df_2017, t2017, 3, 3).std(axis=1).values, 
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "std_7_2017": get_timespan(df_2017, t2017, 7, 7).std(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "std_14_2017": get_timespan(df_2017, t2017, 14, 14).std(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "std_30_2017": get_timespan(df_2017, t2017, 30, 30).std(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "std_60_2017": get_timespan(df_2017, t2017, 60, 60).std(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "std_140_2017": get_timespan(df_2017, t2017, 140, 140).std(axis=1).values,
            
        "oil_mean_3_2017": get_timespan(oil_2017, t2017, 3, 3).mean(axis=1).values,
        "oil_std_3_2017": get_timespan(oil_2017, t2017, 3, 3).std(axis=1).values, 
        "oil_mean_7_2017": get_timespan(oil_2017, t2017, 7, 7).mean(axis=1).values,
        "oil_std_7_2017": get_timespan(oil_2017, t2017, 7, 7).std(axis=1).values,
        "oil_mean_14_2017": get_timespan(oil_2017, t2017, 14, 14).mean(axis=1).values,
        "oil_std_14_2017": get_timespan(oil_2017, t2017, 14, 14).std(axis=1).values,
        "oil_mean_30_2017": get_timespan(oil_2017, t2017, 30, 30).mean(axis=1).values,
        "oil_std_30_2017": get_timespan(oil_2017, t2017, 30, 30).std(axis=1).values,
        "oil_mean_60_2017": get_timespan(oil_2017, t2017, 60, 60).mean(axis=1).values,
        "oil_std_60_2017": get_timespan(oil_2017, t2017, 60, 60).std(axis=1).values,
        "oil_mean_140_2017": get_timespan(oil_2017, t2017, 140, 140).mean(axis=1).values,
        "oil_std_140_2017": get_timespan(oil_2017, t2017, 140, 140).std(axis=1).values,
            
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['std_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').std(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        X['std_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').std(axis=1).values
        
        X['oil_mean_4_dow{}_2017'.format(i)] = get_timespan(oil_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['oil_std_4_dow{}_2017'.format(i)] = get_timespan(oil_2017, t2017, 28-i, 4, freq='7D').std(axis=1).values
        X['oil_mean_20_dow{}_2017'.format(i)] = get_timespan(oil_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        X['oil_std_20_dow{}_2017'.format(i)] = get_timespan(oil_2017, t2017, 140-i, 20, freq='7D').std(axis=1).values
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [27]:
print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

Preparing dataset...


In [28]:
print("Training and predicting models...")

param = {}
param['objective'] = 'reg:linear'
param['eta'] = 0.01
param['max_depth'] = 4
param['silent'] = 1
param['eval_metric'] = 'rmse'
param['min_child_weight'] = 4
param['subsample'] = 0.8
param['colsample_bytree'] = 0.7
param['seed'] = 137
num_rounds = 30000



plst = list(param.items())

MAX_ROUNDS = 30000
val_pred = []
test_pred = []
cate_vars = []

Training and predicting models...


In [29]:
dtest = xgb.DMatrix(X_test)
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = xgb.DMatrix(
        X_train, label=y_train[:, i],
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = xgb.DMatrix(
        X_val, label=y_val[:, i],
        weight=items["perishable"] * 0.25 + 1)
        
    watchlist = [ (dtrain,'train'), (dval, 'val') ]
    model = xgb.train(plst, dtrain, num_rounds, watchlist, early_stopping_rounds=100, verbose_eval=50)
    
    val_pred.append(model.predict(dval, ntree_limit = model.best_ntree_limit))
    test_pred.append(model.predict(dtest, ntree_limit = model.best_ntree_limit))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('xgb_2017_01_06_21_13.csv', float_format='%.4f', index=None)

Step 1
[0]	train-rmse:1.18285	val-rmse:1.13835
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:0.857947	val-rmse:0.824307
[100]	train-rmse:0.696478	val-rmse:0.671171
[150]	train-rmse:0.622162	val-rmse:0.602976
[200]	train-rmse:0.589025	val-rmse:0.573969
[250]	train-rmse:0.57399	val-rmse:0.561783
[300]	train-rmse:0.566539	val-rmse:0.556025
[350]	train-rmse:0.562483	val-rmse:0.552883
[400]	train-rmse:0.559464	val-rmse:0.550321
[450]	train-rmse:0.557518	val-rmse:0.548687
[500]	train-rmse:0.556266	val-rmse:0.547689
[550]	train-rmse:0.555361	val-rmse:0.547046
[600]	train-rmse:0.554668	val-rmse:0.546518
[650]	train-rmse:0.554145	val-rmse:0.546101
[700]	train-rmse:0.553659	val-rmse:0.545742
[750]	train-rmse:0.553201	val-rmse:0.545377
[800]	train-rmse:0.552769	val-rmse:0.545028
[850]	train-rmse:0.552359	val-rmse:0.544718
[900]	train-rmse:0.552017	val-rmse:0.544462
[950]	train-rmse:0.55